### 사전 훈련된 컨브넷 사용하기 (5.3 케라스 창시자에게 배우는 딥러닝 중)

- VGG16 구조는 2014년에 개발
- VGG, ResNet, Inception, Inception-ResNet, Xception 등
- 사전 훈련된 네트워크를 사용하는 방법 두 가지 : 특성 추출(feature extraction), 미세 조정(fine tuning)

In [69]:
from keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet',          # 모델을 초기화할 가중치 체크 포인트(checkpoint) 지정
                  include_top = False,          # 완전 분류기를 포함할 것인가 : ImageNet은 클래스 1000개의 완전 분류기를 포함 -> 완전연결층을 직접 만들 예정
                  input_shape = (150, 150, 3))  # 이 값을 지정하지 않으면 어떤 크기의 입력도 처리 가능함

# include_top이 기본값 Ture이면, 합성곱 층 위에 안전 연결층이 추가되기 때문에,
# input_shape가 원본 모델과 동일한 (224, 224, 3)이 되어야 한다.

conv_base.trainable = False

# include_top :  전체 VGG16의 마지막 층 -> 분류를 담당하는 곳을 불러올지 말지를 결정하는 옵션
# 여기서는 직접 만든 로컬 네트워크를 연결할 것이므로
# 또한 불러올 부분은 새롭게 학습되는 것이 아니므로 -> 학습되지 않도록 .trainable 옵션을 False로 설정

In [70]:
conv_base.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 150, 150, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 150, 150, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 75, 75, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 75, 75, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 75, 75, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 37, 37, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 37, 37, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 18, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 18, 18, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 9, 9, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 4, 4, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [71]:
# 최종 특성 맵의 크기는 (4, 4, 512) -> 이 특성 위에 완전 연결 층을 추가

from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [72]:
import keras
import tensorflow

print(keras.__version__)
print(tensorflow.__version__)

3.2.0
2.16.1


In [79]:
from tensorflow.keras.layers import Dense, Flatten

model = models.Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

In [80]:
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 4, 4, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_17 (Flatten)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [81]:
model.build(input_shape=(None, 150, 150, 3))    # input의 형태를 입력해줘야 정확한 Param 갯수가 출력된다.
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 4, 4, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_17 (Flatten)                 │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 64)                  │         524,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,239,170 (58.13 MB)

 Trainable params: 524,482 (2.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [76]:
# len(model.trainable_weights)  # conv_base를 동결하기 전 훈련되는 가중치이 수

In [20]:
# conv_base의 가중치를 동결

# conv_base.trainable = False
# len(model.trainable_weights)

0

In [ ]:
# Non-trainable params: 14,714,688 (56.13 MB)
# VGG Net의 파라미터는 학습이 진행되지 않음 -> 기 학습된 값을 그대로 사용
# 최종 분류를 위해 추가된 Layer에 대해서만 새롭게 학습
# : 평탄화 + 분류(2개) 방식의 문제로 축소


# 다양한 모델을 호출하는 방법
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.inception_v3 import inceptionV3
# from tensorflow.keras.applications.resnet import ResNet50
# from tensorflow.keras.applications.vgg19 import VGG19

# model_VGG16 = VGG16(weghts = 'imagenet')
# model_VGG19 = VGG19(weghts = 'imagenet')
# model_RESNET50 = ResNet50(weghts = 'imagenet')
# model_INCEPTIONV3 = InceptionV3(weghts = 'imagenet')

### 전이학습(Transfer Learning)
- 기존의 학습 결과를 가져와서 유사한 프로젝트에 사용하는 방법
- 진행 순서
1) 대규모 데이터셋에서 학습된 기존의 네트워크 불러오기 : VGGNet - VGG16
2) CNN 모델의 앞쪽을 이 네트워크로 채우기
3) 뒤쪽 레이어에서 내 프로젝트와 연결하기
4) 이 두 네트워크가 잘 맞물리게 미세 조정 (fine tunning)

In [ ]:
# 예를들어 "뇌 MRI" 사진을 판독하는 문제의 경우,
# 뇌 사진과 관련이 없는 수백만 장의 이미지넷 학습정보가 큰 역할을 하는 이유는,
# '형태'를 구분하는 기본적인 학습이 되어 있기 때문

# 딥러닝은 학습이 시작되면 어떤 픽셀의 조합이 '선'이고 어떤 형태의 그룹이 '면'이 되는지부터 파악해야 한다.
# 아무런 정보도 없이 MRI 사진 판별을 시작한다면, 이러한 기본적인 정보를 얻어내는 데도 
# 많은 시간을 쏟아야 한다. 전이학습은 이런 부분을 해결해준다.

# 출처 : https://m.blog.naver.com/mario002